# Python 知识

## 描述符和 <class 'function'>

考虑：
```python
class MyDescriptor:
    def __init__(self):
        pass
    def __get__(self, instance, owner):
        return "描述符返回值"
    
def user_func(x):
    pass
```

- 含有 `__get__` 的类型称为 ***描述符***。对于一个描述符实例 `x`，`*.x` 等价为
    ```python
    x.__get__(*, type(*))
    ```

- python 中 ***自定义def*** 都是 `<class 'function'>` 类型的。该类型是由 C 层面实现的，其等价实现为
    ```python
    class Function:
    def __init__(self, func_code, func_name):
        self.__code__ = func_code
        self.__name__ = func_name

    # 绑定
    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        else:
            import types
            return types.MethodType(self, instance)
    
    # 调用
    def __call__(self, *args, **kwargs):
        pass
    ```


    ```python
    print(type(MyDescriptor.__init__)) # <class 'function'>
    print(type(MyDescriptor.__get__))  # <class 'function'>
    print(type(user_func)) # <class 'function'>
    ```
    即 `<class 'function'>` 类型是描述符。

    - `<class 'function'>` 的 `__get__` 由 C 语言实现，不包含 `__get__` 即不是描述符。
        ```python
        print(type(MyDescriptor.__get__.__get__))  # <class 'method-wrapper'>
        print(type(MyDescriptor.__init__.__get__)) # <class 'method-wrapper'>
        print(type(user_func.__get__))  # <class 'method-wrapper'>
        ```

## 将 `function` 绑定为 `method`
使用 `types.MethodType` 可以将 `<class 'function'>` 变成 `<class 'method'>`。
`<class 'method'>` 的部分等价实现：
```python
class Method:
    
    def __init__(self, func, instance):
        self.__func__ = func      # 原始函数
        self.__self__ = instance  # 绑定的实例
    
    def __call__(self, *args, **kwargs):
        return self.__func__(self.__self__, *args, **kwargs)
```

例子：
```python
def standalone_function(self, x):
    return f"处理 {x} 来自 {self}"

class MyClass:
    def __init__(self, name):
        self.name = name

obj = MyClass("测试对象")

print(f"{type(standalone_function)}")  # <class 'function'>

# 绑定到实例
instance_method = types.MethodType(standalone_function, obj)
print(f"{type(instance_method)}")  # <class 'method'>

# 绑定到类
class_method = types.MethodType(standalone_function, MyClass)
print(f"{type(class_method)}")  # <class 'method'>

# 调用对比
print(f"{standalone_function(obj, 'data1')}") # 处理 data1 来自 <__main__.MyClass object at 0x00000233633C7230>
print(f"{instance_method('data2')}")  # 处理 data2 来自 <__main__.MyClass object at 0x00000233633C7230>
print(f"{class_method('data3')}")  # 处理 data3 来自 <class '__main__.MyClass'>
```

## `classmethod` 的Python等价实现
根据 Python 官方文档，`classmethod` 的纯Python等价实现是：
```python
class ClassMethod:
    def __init__(self, f):
        self.__func__ = f

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        # 如果 self.f 是个描述符即 function 对象，用其自身的 __get__ 绑定
        if hasattr(type(self.__func__), '__get__'):
            return self.__func__.__get__(cls, type(cls))
        return MethodType(self.__func__, cls)
```
即 `classmethod` 的核心思想当使用 `__get__` 对自身存储的 `self.__func__` 进行绑定时，一定是绑定到类而非实例上。
- 考虑
    ```python
    @classmethod
    def func():...
    ```
    等价于
    ```python 
    func = classmethod(func)    # 构建描述符对象（func 拥有了 self.__func__　属性）
    ```
- 而
    ```python
    Class/obj.func()
    ```
    等价于两步：
    ```python 
    func.__get__(None/obj, Class/type(obj)).__call__() # 先绑定再运行
    ```

## `staticmethod` 的Python等价实现
根据 Python 官方文档，`staticmethod` 的纯Python等价实现是：
```python
class staticmethod:
    def __init__(self, func):
        self.func = func
    
    def __get__(self, instance, owner):
        return self.func
```

## 描述符协议

```python
class Descriptor:

    def __init__(self):
        ...

    def __get__(self, instance, owner):
        ...

    def __set__(self, instance, value):
        ...

    def __delete__(self, instance):
        ...

class User:
    descriptor = Descriptor()

user = User()
```

当类似于 `= user.descriptor` 使用时，实际上是触发了 `descriptor.__get__(user, User)`。

`user.descriptor = 值` 实际上触发了 `descriptor.__set__(user, 值)`。

`del user.descriptor` 实际上触发了 `descriptor.__delete__(user)`。

### property 类的 Python 等价实现

**注意 `porperty` 是由 C 层面实现的，所以 `__get__`、`__set__`、`__delete__` 都不是描述符！**

```python
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc"

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError("unreadable attribute")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(obj)

    # 返回新的 Property 实例
    def getter(self, fget):
        return type(self)(fget, self.fset, self.fdel, self.__doc__)

    def setter(self, fset):
        return type(self)(self.fget, fset, self.fdel, self.__doc__)

    def deleter(self, fdel):
        return type(self)(self.fget, self.fset, fdel, self.__doc__)
```

### @property

使用方法如下

```python
class MyClass:
    def __init__(self):

    # value = property(函数 value(self))
    @property
    def value(self):
        ...

    # value = value.setter(函数 value(self, new_value))
    @value.setter
    def value(self, new_value):
        ...

    # value = value.setter(函数 value(self))
    @value.deleter
    def value(self):
        ...
```

所以相当于创建了三次 property 实例，最后该实例成为 MyClass 的一个类属性 value。

然后 MyClass 或其实例就可以按照 `描述符协议` 来使用 value。

## `a.b`/`getattr(a, b)` 的内部过程
- (1) 在类层次结构 `type(a).__mro__` 中查找是否有名为 `b` 的数据描述符。

    `数据描述符`：同时实现了 `__get__` 和 `__set__` 方法的对象。这种情况下，`a.b` 相当于 
    ```python
    b.__get__(a, type(a))
    ```
    如果 `__get__` 是自定义的即是 `<class 'function'>`，那么实际上发生了
    ```python
    __get__.__get__(b, type(b))(a, type(a))
    ```

    其中第二个 `__get__` 是 C 层面定义的，所以实际上相当于

    ```python
    __get__(__get__, b, type(b))(a, type(a)) 
    ```
- (2) 查找实例字典
    ```python
    if hasattr(a, '__dict__') and 'b' in a.__dict__:
        return a.__dict__['b']
    ```
- (3) 查找非数据描述符和类属性
    - `非数据描述符`：只实现了 `__get__` 方法（如函数、方法、property等）。

    - `普通类属性`：存储在类的 `__dict__` 中的普通值

    如果找到描述符，调用其 `__get__(a, type(a))` 方法；如果是普通属性，直接返回。
- (4) 调用 `__getattr__` 方法

    如果以上步骤都没有找到属性，且对象定义了 `__getattr__` 方法，会调用：
    ```python
    a.__getattr__('b')
    ```
- (5) 抛出 `AttributeError`

    如果所有步骤都失败，抛出 `AttributeError` 异常。

# 代码分析

## class class_or_instancemethod(classmethod)
用来装饰函数：装饰后的函数被引用时**根据调用者是类型/实例，将函数绑定到类型/实例上**。
- 考虑
    ```python
    @class_or_instancemethod
    def func():...

    Class/obj.func()
    ```
    其中 `Class/obj.func` 等价于
    ```python
    func.__get__(None/obj, Class/type(obj))
    ```

```python
class class_or_instancemethod(classmethod):
    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        descr_get = super().__get__ if instance is None else self.__func__.__get__
        return descr_get(instance, owner)
```

## class classproperty(object)
用来装饰函数：装饰后的函数**被引用时就相当于被调用，参数是引用者的类型**。
- 考虑
    ```python
    @classproperty
    def func():...
    ```
    则 `*.func` 等价于 `func(type(*))`

```python
class classproperty(object):
    def __init__(self, func: tp.Callable) -> None:
        self.func = func
        self.__doc__ = getattr(func, '__doc__')

    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        return self.func(owner)

    def __set__(self, instance: object, value: tp.Any) -> None:
        raise AttributeError("can't set attribute")
```

## class class_or_instanceproperty(object)
用来装饰函数：装饰后的函数**被引用时就相当于被调用，参数就是引用者**。
- 考虑
    ```python
    @classproperty
    def func():...
    ```
    则 `*.func` 等价于 `func(*)`
```python
class class_or_instanceproperty(object):
    def __init__(self, func: tp.Callable) -> None:
        self.func = func
        self.__doc__ = getattr(func, '__doc__')

    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        if instance is None:
            return self.func(owner)
        return self.func(instance)

    def __set__(self, instance: object, value: tp.Any) -> None:
        raise AttributeError("can't set attribute")
```

## class custom_property
用来装饰函数：
- 首先，创建实例 `instance = MyClass(*args, **kwargs)` 的过程为
    ```python
    instance = MyClass.__new__(MyClass, *args, **kwargs)
    if isinstance(instance, MyClass):
        instance.__init__(*args, **kwargs)
    ```

- 两种装饰方式
  - 不带参数
    ```python
    @custom_property
    def func(): ...
    ```
    相当于 ``func = custom_property(func)``
  - 带参数
    ```python
    @custom_property(**kwargs)
    def func(): ...
    ```
    相当于 `func = custom_property(**kwargs)(func)`。
    其中 `custom_property(**kwargs)` 会先调用 `__new__`，但是返回的不是 `custom_property` 的实例而是 `lambda` 表达式。
    所以先不会调用 `__init__`，而是接收 `func`，然后重新构建实例。
    
使用方式
- `obj.func`：等价于 `func(obj)`
- `Class.func`：返回该描述符本身
- `func()`：pass，待实现

```python
class custom_property:
    def __new__(cls: tp.Type[custom_propertyT], *args, **flags) -> tp.Union[tp.Callable, custom_propertyT]:
        if len(args) == 0:  
            return lambda func: cls(func, **flags)
        elif len(args) == 1:
            return super().__new__(cls)
        raise ValueError("Either function or keyword arguments must be passed")

    def __init__(self, func: tp.Callable, **flags) -> None:
        self.func = func 
        self.name = func.__name__  
        self.flags = flags  
        self.__doc__ = getattr(func, '__doc__')  

    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        if instance is None:
            return self
        return self.func(instance)

    def __set__(self, instance: object, value: tp.Any) -> None:
        raise AttributeError("can't set attribute")

    def __call__(self, *args, **kwargs) -> tp.Any:
        pass  
```

## def should_cache

### class CacheCondition(tp.NamedTuple)
```python
class CacheCondition(tp.NamedTuple):
    instance: tp.Optional[object] = None
    func: tp.Optional[tp.Union[tp.Callable, "cached_property", str]] = None
    cls: tp.Optional[tp.Union[type, str]] = None
    base_cls: tp.Optional[tp.Union[type, str]] = None
    flags: tp.Optional[dict] = None
    rank: tp.Optional[int] = None
```

### should_cache._get_condition_rank
函数 `_get_condition_rank` 的设计：
- 使用参数 `cond` 去匹配 `should_cache` 的参数 `func_name, instance, func, **flags`
  - `cond` 作为 `CacheCondition`，参与匹配的是 `cond` 的所有非 `None` 项
  - 如果未通过返回 `100`
- 根据 `cond.rank` 是否存在，生成优先级列表 `[cond.rank, cond.rank, ...]` 或是 `[0, 1, ...]`（12个元素）。
- 根据 `cond` 的所有非 `None` 项（此时说明都通过了匹配），从优先级列表中返回对应优先级
  - instance + func：针对特定实例的特定方法/属性
  - instance + flags：针对特定实例的带特定标志的方法/属性
  - instance：针对特定实例的所有方法/属性
  - cls + func：针对特定类的特定方法/属性
  - cls + flags：针对特定类的带特定标志的方法/属性
  - cls：针对特定类的所有方法/属性
  - base_cls + func：针对基类的特定方法/属性
  - base_cls + flags：针对基类的带特定标志的方法/属性
  - base_cls：针对基类的所有方法/属性
  - func + flags：针对带特定标志的特定方法/属性
  - func：针对特定方法/属性
  - flags：针对带特定标志的所有方法/属性

```python
def should_cache(func_name: str, instance: object, func: tp.Optional[tp.Callable] = None, **flags) -> bool:
    from vectorbt._settings import settings
    caching_cfg = settings['caching']

    start_rank = 100

    def _get_condition_rank(cond: CacheCondition) -> int:
        checks.assert_instance_of(cond, CacheCondition)

        if cond.instance is not None:
            if instance is not cond.instance:
                return start_rank
            
        if cond.func is not None:
            if isinstance(cond.func, cached_property):  
                if func != cond.func.func:
                    return start_rank
            elif callable(cond.func) and hasattr(func, 'func') and hasattr(cond.func, 'func'):  
                if func.func != cond.func.func:
                    return start_rank
            elif isinstance(cond.func, str):
                if func_name != cond.func:
                    return start_rank
            else:
                raise TypeError(f"Caching condition {cond}: func must be either a callable or a string")
                
        if cond.cls is not None:
            if inspect.isclass(cond.cls):
                if type(instance) != cond.cls:
                    return start_rank
            elif isinstance(cond.cls, str):
                if type(instance).__name__ != cond.cls:
                    return start_rank
            else:
                raise TypeError(f"Caching condition {cond}: cls must be either a class or a string")
                
        if cond.base_cls is not None:
            if inspect.isclass(cond.base_cls) or isinstance(cond.base_cls, str):
                if not checks.is_instance_of(instance, cond.base_cls):
                    return start_rank
            else:
                raise TypeError(f"Caching condition {cond}: base_cls must be either a class or a string")
                
        if cond.flags is not None:
            if not isinstance(cond.flags, dict):
                raise TypeError(f"Caching condition {cond}: flags must be a dict")
            for k, v in cond.flags.items():
                if k not in flags or flags[k] != v:
                    return start_rank
                    
        if cond.rank is not None:
            if not isinstance(cond.rank, int):
                raise TypeError(f"Caching condition {cond}: rank must be an integer")
            ranks = [cond.rank for _ in range(12)]
        else:
            ranks = list(range(12))

        if cond.instance is not None and cond.func is not None:
            return ranks[0]
        if cond.instance is not None and cond.flags is not None:
            return ranks[1]
        if cond.instance is not None:
            return ranks[2]
        if cond.cls is not None and cond.func is not None:
            return ranks[3]
        if cond.cls is not None and cond.flags is not None:
            return ranks[4]
        if cond.cls is not None:
            return ranks[5]
        if cond.base_cls is not None and cond.func is not None:
            return ranks[6]
        if cond.base_cls is not None and cond.flags is not None:
            return ranks[7]
        if cond.base_cls is not None:
            return ranks[8]
        if cond.func is not None and cond.flags is not None:
            return ranks[9]
        if cond.func is not None:
            return ranks[10]
        if cond.flags is not None:
            return ranks[11]

        return start_rank
```

### def should_cache
注意 `settings['caching']`：
```python
caching=dict(
    enabled=True,  
    whitelist=[  
        CacheCondition(base_cls=ArrayWrapper), 
        CacheCondition(base_cls=ColumnGrouper),  
        CacheCondition(base_cls=ColumnMapper) 
    ],
    blacklist=[]  
)
```

函数 `should_cache` 的设计：
- 使用 `settings['caching']['whitelist']/['blacklist']` 中的各 `CacheCondition` 去匹配参数 `func_name, instance, func, **flags`
  - 计算最小值也就是最大优先级 `white_rank` 和 `black_rank`
- 然后确定是否需要缓存：
  - 如果 `white_rank == black_rank`，返回 `settings['caching']['enabled']`
  - 否则，返回 `white_rank < black_rank`

```python
def should_cache(func_name: str, instance: object, func: tp.Optional[tp.Callable] = None, **flags) -> bool:
    from vectorbt._settings import settings
    caching_cfg = settings['caching']

    start_rank = 100

    def _get_condition_rank(cond: CacheCondition) -> int: ...

    white_rank = start_rank
    if len(caching_cfg['whitelist']) > 0:
        for cond in caching_cfg['whitelist']:
            white_rank = min(white_rank, _get_condition_rank(cond))

    black_rank = start_rank
    if len(caching_cfg['blacklist']) > 0:
        for cond in caching_cfg['blacklist']:
            black_rank = min(black_rank, _get_condition_rank(cond))

    if white_rank == black_rank:  
        return caching_cfg['enabled']  
    return white_rank < black_rank
```

## class cached_property(custom_property)
用来装饰函数，继承了 `custom_property`，相较于前者多了将引用结果放置在被引用实例的 `__dict__` 中的功能。
- 两种装饰方式
  - 不带参数
      ```python
      class A
          @cached_property
          def func(): ...
      ```
      等价于 `func = cached_property(func)`
  - 带参数
      ```python
      class
      @custom_property(**kwargs)
      def func(): ...
      ```
      等价于 `func = cached_property(**kwargs)(func)`
- `__set_name__`
  - **当一个描述符实例被创建时，会自动调用 `__set_name__`（默认实现为 `pass`）**。例如
    ```python
    class MyDescriptor:
        def __set_name__(self, owner, name):...  
        def __get__(self, instance, owner):...

    class MyClass:
        # 该行代码执行时，Python 自动调用 my_attr.__set_name__(MyClass, 'my_attr')
        my_attr = MyDescriptor()
    ```
  - 对于本类，如上面的 `func = cached_property(func)`，相当于
    ```python
    func.name = 'func'
    ```   

使用方式
- `obj.func`：即 `func.__get__(obj, type(obj))`
  - 判断该描述符是否应当被缓存到被引用实例的 `__dict__` 中：
    - 使用`should_cache`：`settings['caching']['whitelist']/['blacklist']` 匹配 `self.name, obj, self.func, self.flags`
  - 如果未通过，相当于 `self.func(obj)`
  - 否则，从该实例的 `__dict__` 查找之前的被引用结果
      - 从 `obj.__dict__` 中查找键为 `__cached_ + self.name` 的键值对，如果没有找到
        - `obj.__dict__[__cached_ + self.name]` 存入 `(__cached_ + self.name: self.func(obj))`
      - 如果找到，则直接返回对应的值即 `self.fun(obj)`
- `Class.func`：返回该描述符本身
- `func()`：pass，待实现

```python
class cached_property(custom_property):
    def __init__(self, func: tp.Callable, **flags) -> None:
        super().__init__(func, **flags)
        self.lock = RLock()

    def clear_cache(self, instance: object) -> None:
        if hasattr(instance, self.attrname):
            delattr(instance, self.attrname)

    @property
    def attrname(self) -> str:
        return '__cached_' + self.name

    def __set_name__(self, owner: tp.Type, name: str) -> None:
        self.name = name

    def __get__(self, instance: object, owner: tp.Optional[tp.Type] = None) -> tp.Any:
        if instance is None:
            return self
        
        if not should_cache(self.name, instance, func=self.func, **self.flags):
            return super().__get__(instance, owner=owner)
        cache = instance.__dict__
        val = cache.get(self.attrname, _NOT_FOUND)
        if val is _NOT_FOUND:
            with self.lock:
                val = cache.get(self.attrname, _NOT_FOUND)
                if val is _NOT_FOUND:
                    val = self.func(instance)
                    cache[self.attrname] = val
        return val

    def __call__(self, *args, **kwargs) -> tp.Any:
        ...
```

## def custom_method
### class custom_methodT(tp.Protocol)
继承自 `Protocol`，这些注解定义了一个**协议**，任何对象如果具有这些属性和方法，就被认为符合 `custom_methodT` 协议：
```python
class custom_methodT(tp.Protocol):
    func: tp.Callable
    flags: tp.Dict

    def __call__(self, *args, **kwargs) -> tp.Any:
        ...
```
### def custom_method
用来装饰函数，装饰后的函数获得了 
- `.func` 属性：存储被装饰的原函数
- `.flags` 属性：存储装饰时的字典参数

两种装饰方式
- 不带参数
    ```python
    @custom_method
    def func(): ...
    ```
    等价于 `func = custom_method(func)`。
- 带参数
    ```python
    @custom_method(flag1=value1, flag2=value2)
    def func(): ...
    ```
    等价于 
    ```
    func = custom_method(flag1=value1, flag2=value2)(func)
            = decorator(func)
    ```

```python
def custom_method(*args, **flags) -> tp.Union[tp.Callable, custom_methodT]:
    def decorator(func: tp.Callable) -> custom_methodT:
        @wraps(func)
        def wrapper(*args, **kwargs) -> tp.Any:
            return func(*args, **kwargs)

        wrapper.func = func
        wrapper.flags = flags

        return wrapper

    if len(args) == 0:
        return decorator
    elif len(args) == 1:
        return decorator(args[0])
    raise ValueError("Either function or keyword arguments must be passed")
```


## def cached_method
### LRU
基础是一个Hash表，put/get/remove 都是 O(1) 平均时间复杂度。

在此基础上，附加一个双向链表，将最近访问(get/put)的元素放在链表头部。当缓存满时，删除链表尾部的元素。
### class cached_methodT(custom_methodT)
```python
class cached_methodT(custom_methodT):
    maxsize: int
    typed: bool
    name: str
    attrname: str
    lock: RLock
    clear_cache: tp.Callable[[object], None]

    def __call__(self, *args, **kwargs) -> tp.Any:
        ...
```
### def cached_method
用来装饰函数，两种装饰方式
- 不带参数
    ```python
    @cached_method
    def func(): ...
    ```
    等价于 `func = cached_method(func)`。
- 带参数
    ```python
    @cached_method(flag1=value1, flag2=value2)
    def func(): ...
    ```
    等价于 
    ```
    func = cached_method(flag1=value1, flag2=value2)(func)
         = decorator(func)
    ```

装饰后的函数获得了 
- `.func`：存储被装饰的原函数
- `.flags`：存储装饰时的字典参数
- `.maxsize`：LRU 缓存的最大大小
- `.typed`：LRU 缓存的类型敏感参数
- `.name`：原函数名称
- `.attrname`：`__cached_ + func.__name__`
- `.lock`
- `.clear_cache`

装饰后的使用方式 `func(obj, *args, **kwargs)`：
- 检查 `obj.__dict__` 是否有名为 `func.name` 的属性，如果有存到 `_func`
- 判断 `func` 是否应当与 `obj.__dict__` 关联：
  - 使用`should_cache`：`settings['caching']['whitelist']/['blacklist']` 匹配 `func.name, obj, _func, func.flags`
  - 如果未通过，返回 `func.func(obj, *args, **kwargs)`
- 应当关联
  - 从 `obj.__dict__` 查找 `__cached_ + func.__name__`。如果没有找到
  - `obj.__dict__[__cached_ + func.name]` 存入 `cached_func = lru_cache 化的 partial_func`
    - 注意
        ```python
        def partial_func(*args, **kwargs) -> tp.Any:
            return func(instance, *args, **kwargs)
        ```
- 检查 `args` 和 `kwargs` 是否都是可哈希的
  - 如果是：调用 `cached_func(*args, **kwargs)`
  - 否则：调用 `func(obj, *args, **kwargs)`

```python
def cached_method(*args, maxsize: int = 128, typed: bool = False,
                  **flags) -> tp.Union[tp.Callable, cached_methodT]:

    def decorator(func: tp.Callable) -> cached_methodT:
        @wraps(func)
        def wrapper(instance: object, *args, **kwargs) -> tp.Any:
            def partial_func(*args, **kwargs) -> tp.Any:
                return func(instance, *args, **kwargs)

            _func = None
            if hasattr(instance, wrapper.name):
                _func = getattr(instance, wrapper.name)
            if not should_cache(wrapper.name, instance, func=_func, **wrapper.flags):
                return func(instance, *args, **kwargs)
            cache = instance.__dict__
            cached_func = cache.get(wrapper.attrname, _NOT_FOUND)
            if cached_func is _NOT_FOUND:
                with wrapper.lock:
                    cached_func = cache.get(wrapper.attrname, _NOT_FOUND)
                    if cached_func is _NOT_FOUND:
                        cached_func = lru_cache(maxsize=wrapper.maxsize, typed=wrapper.typed)(partial_func)
                        cache[wrapper.attrname] = cached_func

            hashable = True
            for arg in args:
                if not checks.is_hashable(arg):
                    hashable = False
                    break
            for k, v in kwargs.items():
                if not checks.is_hashable(v):
                    hashable = False
                    break
            if not hashable:
                return func(instance, *args, **kwargs)
            return cached_func(*args, **kwargs)

        def clear_cache(instance):
            if hasattr(instance, wrapper.attrname):
                delattr(instance, wrapper.attrname)

        wrapper.func = func                             # 存储对原函数的引用
        wrapper.flags = flags                           # 存储装饰器的配置参数
        wrapper.maxsize = maxsize                       # 设置缓存的最大大小
        wrapper.typed = typed                           # 设置是否启用类型敏感缓存
        wrapper.name = func.__name__                    # 存储原函数的名称
        wrapper.attrname = '__cached_' + func.__name__  # 生成缓存属性的名称
        wrapper.lock = RLock()                          # 创建一个可重入锁，用于线程安全的缓存更新
        wrapper.clear_cache = clear_cache               # 存储清除缓存的方法

        return wrapper

    if len(args) == 0:
        return decorator
    elif len(args) == 1:
        return decorator(args[0])
    raise ValueError("Either function or keyword arguments must be passed")
```

## def attach_binary_magic_methods
### binary_magic_config
```python
binary_magic_config = Config(
    {
        '__eq__': dict(func=np.equal),
        '__ne__': dict(func=np.not_equal),
        '__lt__': dict(func=np.less),
        '__gt__': dict(func=np.greater),
        '__le__': dict(func=np.less_equal),
        '__ge__': dict(func=np.greater_equal),
        # arithmetic ops
        '__add__': dict(func=np.add),
        '__sub__': dict(func=np.subtract),
        '__mul__': dict(func=np.multiply),
        '__pow__': dict(func=np.power),
        '__mod__': dict(func=np.mod),
        '__floordiv__': dict(func=np.floor_divide),
        '__truediv__': dict(func=np.true_divide),
        '__radd__': dict(func=lambda x, y: np.add(y, x)),
        '__rsub__': dict(func=lambda x, y: np.subtract(y, x)),
        '__rmul__': dict(func=lambda x, y: np.multiply(y, x)),
        '__rpow__': dict(func=lambda x, y: np.power(y, x)),
        '__rmod__': dict(func=lambda x, y: np.mod(y, x)),
        '__rfloordiv__': dict(func=lambda x, y: np.floor_divide(y, x)),
        '__rtruediv__': dict(func=lambda x, y: np.true_divide(y, x)),
        # mask ops
        '__and__': dict(func=np.bitwise_and),
        '__or__': dict(func=np.bitwise_or),
        '__xor__': dict(func=np.bitwise_xor),
        '__rand__': dict(func=lambda x, y: np.bitwise_and(y, x)),
        '__ror__': dict(func=lambda x, y: np.bitwise_or(y, x)),
        '__rxor__': dict(func=lambda x, y: np.bitwise_xor(y, x))
    },
    readonly=True,
    as_attrs=False
)
```
### def attach_binary_magic_methods
用来装饰类：装饰后的类可以参与预定义的二元操作
- 装饰参数应当是一个函数
  - 接收两个操作数，和一个操作函数
  - 这个函数负责
    - 将操作数转换成操作函数能够使用的格式
    - 使用操作函数计算
    - 将计算结果转换成合适的格式

例如：
```python
def my_translate_func(self, other, numpy_func):
    arr1 = np.asarray(self.data)
    arr2 = np.asarray(other.data if hasattr(other, 'data') else other)
    result = numpy_func(arr1, arr2)
    return MyClass(result)

@attach_binary_magic_methods(my_translate_func)
class MyClass:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return f"MyClass({self.data})"

obj1 = MyClass([1, 2, 3])
obj2 = MyClass([4, 5, 6])

result_add = obj1 + obj2      # 调用 __add__ -> [5, 7, 9]
result_mul = obj1 * 2         # 调用 __mul__ -> [2, 4, 6]
result_pow = obj1 ** 2        # 调用 __pow__ -> [1, 4, 9]
```

```python
def attach_binary_magic_methods(translate_func: BinaryTranslateFuncT,
                                config: tp.Optional[Config] = None) -> WrapperFuncT:
    if config is None:
        config = binary_magic_config

    def wrapper(cls: tp.Type[tp.T]) -> tp.Type[tp.T]:
        for target_name, settings in config.items():
            func = settings['func']
            def new_method(self,
                           other: tp.Any,
                           _translate_func: BinaryTranslateFuncT = translate_func,
                           _func: tp.Callable = func) -> tp.SeriesFrame:
                return _translate_func(self, other, _func)

            new_method.__qualname__ = f"{cls.__name__}.{target_name}"
            new_method.__name__ = target_name
            # 相当于：cls.__dict__[target_name] = new_method
            setattr(cls, target_name, new_method)
        return cls
    return wrapper
```

## def attach_unary_magic_methods
### unary_magic_config
```python
unary_magic_config = Config(
    {
        '__neg__': dict(func=np.negative),
        '__pos__': dict(func=np.positive),
        '__abs__': dict(func=np.absolute),
        '__invert__': dict(func=np.invert)
    },
    readonly=True,
    as_attrs=False
)
```
### def attach_unary_magic_methods
用来装饰类：装饰后的类可以参与预定义的一元操作

```python
UnaryTranslateFuncT = tp.Callable[[tp.Any, tp.Callable], tp.Any]

def attach_unary_magic_methods(translate_func: UnaryTranslateFuncT,
                               config: tp.Optional[Config] = None) -> WrapperFuncT:
    if config is None:
        config = unary_magic_config

    def wrapper(cls: tp.Type[tp.T]) -> tp.Type[tp.T]:
        for target_name, settings in config.items():
            func = settings['func']

            def new_method(self,
                           _translate_func: UnaryTranslateFuncT = translate_func,
                           _func: tp.Callable = func) -> tp.SeriesFrame:
                return _translate_func(self, _func)

            new_method.__qualname__ = f"{cls.__name__}.{target_name}"
            new_method.__name__ = target_name
            setattr(cls, target_name, new_method)
        return cls

    return wrapper
```